In [ ]:
import pandas as pd
import os
import psycopg2
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

: 

In [2]:
# Cria conexão com o banco de dados
def conexao():
    con = psycopg2.connect(
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        database=os.getenv("POSTGRES_DB"),
        host="localhost",
    )

    return con

# Cria função para selecionar os dados
def se_dados(query):
    vcon = conexao()
    c = vcon.cursor()
    c.execute(query)
    rows = c.fetchall()
    vcon.commit()
    vcon.close()
    return rows                    

In [4]:
# Cria query para gerar o relatório
query = f" SELECT DISTINCT papel, div_liquida, cotacao, pl, vpa, ev_ebitda, marg_liquida, lucro_liquido_12m, lucro_liquido_3m \
            FROM dados \
                WHERE data_dado_inserido >= '07-12-2022' \
                    AND div_liquida < '0' \
                    AND pl > 5 \
                    AND pl < 15 \
                        ORDER BY 2 "
result = se_dados(query)

# Cria DataFrame e salva o arquivo gerado em csv
df_rl = pd.DataFrame(result)
# Renomea o nome das colunas do Dataframe
df_rl.rename(columns={0:'Papel', 
    1:'Dívida Líquida', 
    2:'Cotação',
    3:'PL',
    4:'VPA',
    5:'EV/Ebitda',
    6:'Margem Líquida',
    7:'Lucro Líquido 12m',
    8:'Lucro Líquido 3m'}, inplace=True)
df_rl.to_csv('../Analise/relatorio.csv', sep=';', encoding='ISO-8859-1')
df_rl.head(7)

,Papel,Dívida Líquida,Cotação,PL,VPA,EV/Ebitda,Margem Líquida,Lucro Líquido 12m,Lucro Líquido 3m
0,PSSA3,"-R$ 9.927.950.000,00","R$ 22,99",13.37,15.39,0.23,4.3,"R$ 1.112.040.000,00","R$ 272.736.000,00"
1,CMIN3,"-R$ 1.536.390.000,00","R$ 3,76",7.41,2.40,3.74,23.3,"R$ 2.782.520.000,00","R$ 514.057.000,00"
2,BRGE12,"-R$ 1.391.210.000,00","R$ 9,90",9.28,17.52,0.97,2917.1,"R$ 78.529.000,00","R$ 52.037.000,00"
3,BRGE3,"-R$ 1.391.210.000,00","R$ 9,71",9.11,17.52,0.99,2917.1,"R$ 78.529.000,00","R$ 52.037.000,00"
4,BRGE5,"-R$ 1.391.210.000,00","R$ 11,53",10.81,17.52,0.80,2917.1,"R$ 78.529.000,00","R$ 52.037.000,00"
5,BRGE6,"-R$ 1.391.210.000,00","R$ 12,01",11.26,17.52,0.74,2917.1,"R$ 78.529.000,00","R$ 52.037.000,00"
6,BRGE8,"-R$ 1.391.210.000,00","R$ 8,97",8.41,17.52,1.07,2917.1,"R$ 78.529.000,00","R$ 52.037.000,00"


In [5]:
import win32com.client as win32
import pathlib

# Inserindo arquivo a ser anexado
anexo = pathlib.Path('../Analise/relatorio.csv')
anexo_ab = str(anexo.absolute())

# Criar a integração com o outlook
outlook = win32.Dispatch('outlook.application')

# Criar um email
email = outlook.CreateItem(0)

# Configurar as informações do seu e-mail
email.To = os.getenv("EMAIL")
email.Subject = " Relatório - Ações listadas na Bolsa "
email.HTMLBody = f"""

<p>Olá,</p>
<br>
<p>Segue em anexo o relatório solicitado.</p>
<br>
<p>Atenciosamente,</p>
<p>Lucas Alves.</p>
<p>Analista de Dados</p>
<br>
<img src="../Image/1-relatorio-gerencial.jpg">

"""
# Configurar anexo
email.Attachments.Add(anexo_ab)

# Envia o e-mail
email.Send()
print("Email Enviado")

Email Enviado
